In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Activation, Concatenate
from keras import Sequential

from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import SGD,Adam
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

#Different Transfer learning rate/models
#https://keras.io/api/applications/
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

train_dir = 'grayscale'

img_size = (100, 100)
batch_size = 32

num_classes = 14

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        subset='training',
        shuffle=True,
        seed=42)

val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        subset='validation',
        shuffle=True,
        seed=42)

#base_model = VGG19(weights=None, include_top=False, input_shape=(img_size[0], img_size[1], 1))
#base_model = InceptionV3(include_top = False, weights =None, input_shape = (img_size[0], img_size[1], 1))
#base_model = ResNet50(include_top = False, weights =None, input_shape = (img_size[0], img_size[1], 1)) #
base_model = ResNet101(include_top = False, weights =None, input_shape = (img_size[0], img_size[1], 1))
#base_model = Xception(include_top = False, weights =None, input_shape = (img_size[0], img_size[1], 1))
#base_model = VGG16(include_top = False, weights = None, input_shape = (img_size[0], img_size[1], 1))


for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size)



Found 17800 images belonging to 14 classes.
Found 0 images belonging to 14 classes.
Epoch 1/100


C:\Users\Razaq\AppData\Local\Temp\ipykernel_2828\1046726178.py:90: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


556/556 [==============================] - 345s 612ms/step - loss: 1.9558 - accuracy: 0.3852
Epoch 2/100
556/556 [==============================] - 333s 599ms/step - loss: 1.5472 - accuracy: 0.4818
Epoch 3/100
556/556 [==============================] - 332s 597ms/step - loss: 1.3703 - accuracy: 0.5124
Epoch 4/100
556/556 [==============================] - 334s 600ms/step - loss: 1.2979 - accuracy: 0.5297
Epoch 5/100
556/556 [==============================] - 332s 597ms/step - loss: 1.2676 - accuracy: 0.5275
Epoch 6/100
556/556 [==============================] - 335s 602ms/step - loss: 1.2566 - accuracy: 0.5322
Epoch 7/100
556/556 [==============================] - 336s 604ms/step - loss: 1.2793 - accuracy: 0.5128
Epoch 8/100
556/556 [==============================] - 330s 593ms/step - loss: 1.2628 - accuracy: 0.5203
Epoch 9/100
556/556 [==============================] - 327s 589ms/step - loss: 1.2396 - accuracy: 0.5273
Epoch 10/100
556/556 [==============================] - 328s 589ms/

In [2]:
Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

C:\Users\Razaq\AppData\Local\Temp\ipykernel_2828\753440171.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
target_names = list(train_generator.class_indices.keys())
print('Classification Report:')
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))
print('Confusion Matrix:')
print(confusion_matrix(validation_generator.classes, y_pred))

In [ ]:
# Plot the accuracy and loss graphs for the model
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()
plt.show()